In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import random
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import cv2
from sklearn.model_selection import train_test_split

np.random.seed(0)

# Load data into dataframe
training_data_folder = 'asl_alphabet_train'
data = []

for folder in sorted(os.listdir(training_data_folder)):
    sub_folder = os.path.join(training_data_folder,folder)
    files = [{'label':folder,'path':os.path.join(sub_folder, f)} for f in os.listdir(sub_folder) if os.path.isfile(os.path.join(sub_folder, f))]
    data += files

df = pd.DataFrame(data)

train_dir="asl_alphabet_train"
labels_dict,map_characters = {},{}
order_list=[]
for i in range(26):
    c=chr(ord('A')+i)
    labels_dict[c]=i
    map_characters[i]=c
    order_list+=c
    
from sklearn.preprocessing import LabelBinarizer
lb=LabelBinarizer()

hotEncodedLabels=lb.fit_transform(order_list)
    
# df.to_csv("labelPath.csv",index=False)
# df2 = pd.read_csv('labelPath.csv')
# df2.head()



2022-12-01 11:46:56.572718: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 11:46:58.642563: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-01 11:46:58.642740: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-01 11:47:01.669766: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

Instructions for updating:
non-resource variables are not supported in the long term


In [117]:
def load_data():
    """
    Loads data and preprocess. Returns train and test data along with labels.
    """
    images = []
    labels = []
    size = 64,64
    print("LOADING DATA FROM : ",end = "")
    for folder in os.listdir(train_dir):
        print(folder, end = ' | ')
        for image in os.listdir(train_dir + "/" + folder): #/mnt/d/Zaken/Desktop/computer science/DeepLearningClassificationASL/asl_alphabet_train/A/A1.jpg
            save=train_dir + '/' + folder + '/' + image
            temp_img = cv2.imread(train_dir + '/' + folder + '/' + image, 0) # greyscale
            temp_img = cv2.resize(temp_img, size)
            temp_img = temp_img.flatten()
            images.append(temp_img)
            labels.append(hotEncodedLabels[labels_dict[folder]])
    
    images = np.array(images)
    images = images.astype('float32')/255.0
    
    X_train, X_test, Y_train, Y_test = train_test_split(images, labels, test_size = 0.3, random_state=42,shuffle=True)
    X_test, X_validation, Y_test, Y_validation = train_test_split(X_test, Y_test, test_size = 0.8,random_state=42,shuffle=True)
    print()
    print('Loaded', len(X_train),'images for training,','Train data shape =',X_train.shape)
    print('Loaded', len(X_validation),'images for validation','validation data shape =',X_validation.shape)
    print('Loaded', len(X_test),'images for testing','Test data shape =',X_test.shape)

    return X_train, X_test, Y_train, Y_test, X_validation, Y_validation

X_train, X_test, Y_train, Y_test, X_validation, Y_validation= load_data()




LOADING DATA FROM : A | B | C | D | E | F | G | H | I | J | K | L | M | N | O | P | Q | R | S | T | U | V | W | X | Y | Z | 
Loaded 54600 images for training, Train data shape = (54600, 4096)
Loaded 18720 images for validation validation data shape = (18720, 4096)
Loaded 4680 images for testing Test data shape = (4680, 4096)


In [125]:

categories = 26
features= 4096
x = tf.placeholder(tf.float32, [None, features])
y_ = tf.placeholder(tf.float32, [None, categories])
W = tf.Variable(tf.zeros([features,categories]))
b = tf.Variable(tf.zeros([categories]))

z = tf.matmul(x, W) +b 
pred = tf.nn.softmax(z) 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(y_, z))
update = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

data_x=np.array(X_train)
data_y=np.array(Y_train)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
k=0

for i in range(10000):
    # k+=1
    # if ((k+1)*1000)>=len(data_x): k=0
    # x_100= data_x[k*1000:(k+1)*1000]
    # y_100= data_y[k*1000:(k+1)*1000]
    sess.run(update, feed_dict = {x:data_x, y_:data_y}) 
    if(i%1000==0):
        print(i)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [126]:
correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: X_test, y_: Y_test}))

0.4491453
